# Funciones de Firma Digital

¿Cómo funciona la firma digital?

Consta de tres funciones:
1. Una funcion `generate_keys()` para generar un par de claves, una secreta y una publica
2. Una funcion `sign(message, secret_key)` para generar la firma de un mensaje
3. Una funcion `verify(message, signature, public_key)` para saber si un mensaje ha sido firmado por quien dice firmarlo.

*Nota: la clave publica, la clave secreta y la firma son numeros que pueden ser expresados en distantas bases: decimal, hexadecimal, base64, etc.*
```
secret_key, public_key = generate_keys()
signature = sign(message, secret_key)
is_valid = verify(message, signature, public_key)
```

Que funcion de firma digital utiliza Bitcoin?

Bitcoin utiliza criptografía de curva elíptica para generar las claves:
- https://en.wikipedia.org/wiki/Elliptic-curve_cryptography

Propiedades de las firmas digitales:
- Las firmas verdaderas se pueden verificar:
`verify(message, signature, public_key) == True`
- Nadie puede falsificar una firma:
alguien que puede ver `pk` y todos los mensajes firmados que quiera no puede falsificar esa firma en un mensaje nuevo


## Veamos como funciona una implementacion en python de la firma digital

Instalamos bitcoinlib:
- [pip install](https://pypi.org/project/bitcoinlib/)
- [documentacion](https://bitcoinlib.readthedocs.io/en/latest/)
- [github repo](https://github.com/1200wd/bitcoinlib)

In [ ]:
!sudo apt install -qq build-essential python3-dev libgmp3-dev > /dev/null
!pip install bitcoinlib -U -q

### Generamos el par de claves: secreta y publica

Este par de claves es unico.

In [ ]:
from bitcoinlib.keys import Key, sign, verify
k = Key()
sk, pk = k.secret, k.public()

print(f"Secret Key: {sk}")
print(f"Public Key: {pk}")

Secret Key: 22599353647577282964278189193355568377322138684067962480166634917755452150041
Public Key: 03200b3f537075f94ece9a8bc5c04d17b6c06b4e1632d3b335e01373daa295143f


### Firmamos un mensaje

In [ ]:
sig = sign(b"Hola mi nombre es Alicia", sk)
print(sig)

304402205ba10069848486512e1518d1c4f2e44b38b83f5e3e7da2b89e966a5de745520a02205b1630550dbd72a58eceddebcc6a7f51d8c021c2b70bc9d966f8c3f4ee9732d401


### Verificamos que ese mensaje ha sido firmado correctamente

Con esta funcion verificamos que quien ha firmado es quien posee la clave secreta que corresponde a la clave public `pk`

In [ ]:
is_valid = verify(b"Hola mi nombre es Alicia", sig, pk)
print(is_valid)

True


#### Ejercicio 1:

Intercambiar dos mensajes (uno firmado correctamente y otro que no) con un compañera/o para que verifique cual de los dos esta firmado correctamente y cual no.

In [ ]:
from bitcoinlib.keys import Key, sign, verify
k = Key()
sk, pk = k.secret, k.public()

print(f"Secret Key: {sk}")
print(f"Public Key: {pk}")

msg1 = b'Este mensaje sera el correcto'
sig = sign(msg1, sk)
print(sig)
# Envio msg,sig,pk correctamente
is_valid = verify(msg1, sig, pk)
print(is_valid)

msg2 = b'Este mensaje sera el incorrecto'
sig = sign(msg2, sk)
print(sig)
msg2_modificado = b'Este mensaje sera el incorrecto.'
# Envio el mensaje modificado
is_valid = verify(msg2_modificado, sig, pk)
print(is_valid)

Secret Key: 32141181897061305939248719836056720905338209942178190460644486087954133075399
Public Key: 0385cbc0375b97d4958fc9262e03e24873f49ac4f2ccfa7ff0fb69d36dcb25db5f
304402205d2c92881173e0fb95170f6301bfe173688f5147567fce1cc0549222a3a4d38d022066b9c8c101da20ba2a79ac5e49342a76afc0f87788f3f95cc6874ae931dd6e3601
True
3044022048fd6db6bcd1d9e55468239cf798cbaeb4fe5afae6f332878909e097062a0cb402207dd84f703fb07ce0de0cb3f79cdd01a4e1e7e2194f9be5d81d39ddfafb4410b201
False


Por una cuestion de performance y seguridad es mejor firmar el hash de un mensaje en vez del mensaje en si.

Supongamos que tenemos un documento de 100 paginas. En vez de firmar dicho documento podemos primero utilizar una funcion criptografica de hash como por ejemplo sha256 y luego firmar el hash de dicho documento ya que como dijimos en la unidad anterior cada documento tendra su hash unico, el hash sirve como un identificador para un documento.

#### Ejercicio 2:

Repetir el Ejercicio 1 pero esta vez firmando el hash en vez del mensaje.

In [ ]:
import hashlib
from bitcoinlib.keys import Key, sign, verify
k = Key()
sk, pk = k.secret, k.public()

print(f"Secret Key: {sk}")
print(f"Public Key: {pk}")

msg1 = 'Este mensaje sera el correcto'
msg_hash1 = hashlib.sha256(msg1.encode()).digest()
sig = sign(msg_hash1, sk)
print(sig)
# Envio hash(msg),sig,pk correctamente
is_valid = verify(msg_hash1, sig, pk)
print(is_valid)

msg2 = 'Este mensaje sera el incorrecto'
msg_hash2 = hashlib.sha256(msg2.encode()).digest()
sig = sign(msg_hash2, sk)
print(sig)
# Envio el hash de otro mensaje
is_valid = verify(msg_hash1, sig, pk)
print(is_valid)

Secret Key: 9864170368127870296833013991316029682036480546584096623318836772867260581673
Public Key: 02b7ad02dd9f08a189165b83de3ac749b90d38c3ef3722ccd251ec529da3597bad
3045022100ad7d28df8b4a0d61426a6a42f2b30f853b8516ab59fdd524a3023c43fd7994a402203708f8ecc561a1e068b765b9cf966850fba63b0036c14bc464277ceeb79b592301
True
30450221009c96d1e3d8ffa8831def5f1540d0b35af47cb37f9fb69df51e0ce286b91eb7cc022029cd9f5d362a16ddb80987f9dd1cf067d8d50f937ad6577012ca6da8bf784f7501
False


## Ejercicio extra:

### Utilizar la funcion de firma digital para subir fotografias a instagram u otra plataforma y que nuestros seguidores puedan confirmar que son nuestras y no subidas por un hacker.

### Pensar posibles soluciones y problemas. Discutir en clase.